In [1]:
def load_list_from_file(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        return [line.strip() for line in file]

In [2]:
lunch = load_list_from_file("lunch.txt")
breakfast = load_list_from_file("breakfast.txt")
dinner = load_list_from_file("dinner.txt")

In [11]:
import requests, json
from bs4 import BeautifulSoup

def food_info(name):
    '''
    This function gives you food information for the given input.

    PARAMETERS
        - name(str): name of Korean food in Korean ex) food_info("김치찌개")
    RETURN
        - res(list): list of dict that containing info for some Korean food related to 'name'
        - res['name'](str): name of food
        - res['ingredients'](str): ingredients to make the food
        - res['recipe'](list[str]): contain recipe in order
    '''
    url = f"https://www.10000recipe.com/recipe/list.html?q={name}"
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
    else: 
        print("HTTP response error :", response.status_code)
        return
    
    food_list = soup.find_all(attrs={'class':'common_sp_link'})
    if not food_list:
        print(f"No recipe found for {name}")
        return
    
    food_id = food_list[0]['href'].split('/')[-1]
    new_url = f'https://www.10000recipe.com/recipe/{food_id}'
    new_response = requests.get(new_url)
    if new_response.status_code == 200:
        html = new_response.text
        soup = BeautifulSoup(html, 'html.parser')
    else: 
        print("HTTP response error :", new_response.status_code)
        return
    
    food_info = soup.find(attrs={'type':'application/ld+json'})
    result = json.loads(food_info.text)
    ingredient = ','.join(result['recipeIngredient'])
    recipe = [result['recipeInstructions'][i]['text'] for i in range(len(result['recipeInstructions']))]
    for i in range(len(recipe)):
        recipe[i] = f'{i+1}. ' + recipe[i]
    
    res = {
        'name': name,
        'ingredients': ingredient,
        'recipe': recipe
    }

    return res

In [7]:
print(lunch)

['김치찌개', '된장찌개', '순두부찌개', '북엇국', '미역국', '감자국', '떡국', '콩나물국', '소고기무국', '시래기국', '오이냉국', '비빔밥', '돌솥비빔밥', '김치볶음밥', '참치마요덮밥', '오므라이스', '카레라이스', '제육볶음', '소불고기', '닭갈비', '오징어볶음', '낙지볶음', '김치찜', '돼지갈비찜', '소갈비찜', '닭볶음탕', '고등어조림', '꽁치조림', '두부조림', '감자조림', '연근조림', '멸치볶음', '진미채볶음', '어묵볶음', '콩자반', '시금치나물', '콩나물무침', '도라지무침', '고사리나물', '잡채', '김밥', '충무김밥', '삼각김밥', '주먹밥', '유부초밥', '계란말이', '계란찜', '두부부침', '감자전', '김치전', '해물파전', '호박전', '깻잎전', '동태찌개', '부대찌개', '알탕', '매운탕', '육개장', '갈비탕', '설렁탕', '삼계탕', '순대국', '돼지국밥', '소고기국밥', '버섯전골', '샤부샤부', '곰탕', '콩비지찌개', '칼국수', '비빔국수', '물냉면', '비빔냉면', '쫄면', '라면', '짜장면', '짬뽕', '우동', '야끼우동', '볶음우동', '떡볶이', '순대볶음', '치킨', '양념치킨', '간장치킨', '닭강정', '탕수육', '깐풍기', '마파두부', '짜장밥', '탕수육덮밥', '치킨마요덮밥', '소고기덮밥', '오징어덮밥', '카츠동', '규동', '텐동', '유린기', '팔보채', '양장피', '새우튀김', '돈가스', '생선가스']


In [8]:
food_info('북엇국')

{'name': '북엇국',
 'ingredients': '북어 2줌,무 적당량,달걀1개,멸치다시마육수 6컵,대파 약간,참기름 1큰술,다진마늘 0.5큰술,참치액 1큰술,새우젓 1큰술',
 'recipe': ['1. 무는 먹기 좋은 크기로 썰어주고. 대파도 송송 썰고',
  '2. 북어는 물에 살짝 담가 물기를 제거하고 팬에 참기름1,참치액1 그리고 북어를 넣고 달달 볶다~',
  '3. 썰어놓은 무를 넣고 볶아주다',
  '4. 멸치다시마육수6컵을 부어주고',
  '5. 다진마늘 0.5',
  '6. 새우젓1 넣고 바글 바글 끓여줍니다. 무와 북어에서 맛이 우러나도록 ~',
  '7. 국이 거의 완성될쯤 (무가 익으면 되겠죠) 달걀1개를 풀어 빙~~둘러 부어줍니다. 이때 숟가락으로 젓지 마시고 그냥 두심 되요.',
  '8. 대파 넣고 한소끔 더 끓여 마무리 달걀 넣고 오랫동안 끓이는것 보다 마지막에 넣어 살짝 끓여 마무리 하는게 달걀이 부드럽고 맛있어요. 기호에 따라 후추가루를 넣어도 좋아요 (부족한 간은 소금으로 해주세요)',
  '9. 북엇국 완성 무도 넣고 달걀도 풀어주고 시원하고 뜨끈한 국 한그릇']}

In [12]:
lunch_recipes = []
failed_recipes = []

for dish in lunch:
    try:
        recipe_info = food_info(dish)
        if recipe_info:
            lunch_recipes.append(recipe_info)
        else:
            failed_recipes.append(dish)
    except Exception as e:
        print(f"Error occurred for {dish}: {e}")
        failed_recipes.append(dish)

# JSON 파일로 저장
with open('lunch_recipes.json', 'w', encoding='utf-8') as f:
    json.dump(lunch_recipes, f, ensure_ascii=False, indent=4)

# 실패한 레시피 리스트 저장
with open('lunch_failed_recipes.json', 'w', encoding='utf-8') as f:
    json.dump(failed_recipes, f, ensure_ascii=False, indent=4)

Error occurred for 시금치나물: 'recipeIngredient'


In [13]:
dinner_recipes = []
failed_recipes = []

for dish in dinner:
    try:
        recipe_info = food_info(dish)
        if recipe_info:
            dinner_recipes.append(recipe_info)
        else:
            failed_recipes.append(dish)
    except Exception as e:
        print(f"Error occurred for {dish}: {e}")
        failed_recipes.append(dish)

# JSON 파일로 저장
with open('dinner_recipes.json', 'w', encoding='utf-8') as f:
    json.dump(dinner_recipes, f, ensure_ascii=False, indent=4)

# 실패한 레시피 리스트 저장
with open('dinner_failed_recipes.json', 'w', encoding='utf-8') as f:
    json.dump(failed_recipes, f, ensure_ascii=False, indent=4)

Error occurred for 시금치나물: 'recipeIngredient'


In [15]:
breakfast_recipes = []
breakfast_failed_recipes = []

for dish in breakfast:
    try:
        recipe_info = food_info(dish)
        if recipe_info:
            breakfast_recipes.append(recipe_info)
        else:
            failed_recipes.append(dish)
    except Exception as e:
        print(f"Error occurred for {dish}: {e}")
        failed_recipes.append(dish)

# JSON 파일로 저장
with open('breakfast_recipes.json', 'w', encoding='utf-8') as f:
    json.dump(breakfast_recipes, f, ensure_ascii=False, indent=4)

# 실패한 레시피 리스트 저장
with open('breakfast_failed_recipes.json', 'w', encoding='utf-8') as f:
    json.dump(breakfast_failed_recipes, f, ensure_ascii=False, indent=4)

Error occurred for 두유: 'recipeIngredient'
Error occurred for 팥죽: 'recipeIngredient'
Error occurred for 검은콩죽: 'recipeIngredient'
Error occurred for 유자차: 'recipeIngredient'
Error occurred for 찐빵: 'NoneType' object has no attribute 'text'
